In [1]:
# 着差をスリット数に変換する
MARGIN_TO_SLITS = Dict(
    "同着" => 0,
    "ハナ" => 3,
    "アタマ" => 6,
    "クビ" => 12,
    "1/2" => 24,    # 1/2馬身
    "3/4" => 30,    # 3/4馬身
    "1" => 33,      # 1馬身
    "1.1/4" => 37,  # 1 1/4馬身
    "1.1/2" => 40,  # 1 1/2馬身 (推定値: 2/10〜3/10秒の中間)
    "1.3/4" => 43,  # 1 3/4馬身 (推定値: 3/10秒)
    "2" => 46,      # 2馬身 (3/10秒相当)
    "2.1/2" => 53,  # 2 1/2馬身 (4/10秒相当)
    "3" => 66,      # 3馬身 (5/10秒相当)
    "3.1/2" => 80,  # 3 1/2馬身 (6/10秒相当)
    "4" => 93,      # 4馬身 (7/10秒相当)
    "5" => 113,     # 5馬身 (8.5/10秒相当)
    "6" => 133,     # 6馬身 (1秒相当)
    "7" => 153,     # 7馬身 (11.5/10秒相当)
    "8" => 173,     # 8馬身 (13/10秒相当)
    "9" => 193,     # 9馬身 (14.5/10秒相当)
    "10" => 213,    # 10馬身 (16/10秒相当)
    "大差" => 226   # 17/10秒以上
)

MARGIN_TO_SLITS["10"]


213

In [2]:
using CSV
using DataFrames
using Statistics

# マージンをスリットに変換する関数
function margin_to_slits(margin)
    get(MARGIN_TO_SLITS, margin, 0)
end

# スリットスコアの正規化関数を修正
function normalize_slits(slits_array)
    # 逆転させる（最小値が最大に、最大値が最小になるように）
    max_slits = maximum(slits_array)
    # 1から引くことで、良い成績（スリット数が少ない）ほど高い値になる
    1.0 .- ((slits_array .- minimum(slits_array)) ./ (maximum(slits_array) - minimum(slits_array)))
end

function standardize_slits(slits_array)
    (slits_array .- mean(slits_array)) ./ std(slits_array)
end

function percentile_rank(slits_array)
    n = length(slits_array)
    ranks = [count(x -> x <= val, slits_array) for val in slits_array]
    (ranks .- 0.5) ./ n * 100
end

function relative_performance(slits_array)
    best_performance = minimum(slits_array)
    best_performance ./ slits_array
end

# オッズの処理関数
function process_odds(odds_array)
    win_probability = 1 ./ odds_array

    # 0-1の範囲に正規化（最も人気のある馬が1に近く、人気のない馬が0に近くなる）
    normalized_probability = (win_probability .- minimum(win_probability)) ./
                           (maximum(win_probability) - minimum(win_probability))

    return (norm_prob = normalized_probability,)
end

function main()
    # CSVファイルの読み込み
    df = CSV.read("../data/race.csv", DataFrame)

    # レース毎にグループ化して処理
    transformed_dfs = []

    for (race_name, race_group) in pairs(groupby(df, :race))
        # マージンをスリットに変換
        slits = [margin_to_slits(m) for m in race_group.margin]

        # スリットスコアの正規化
        norm_slits = normalize_slits(slits)

        # オッズの処理
        odds_processed = process_odds(race_group.odds)

        # 必要な列のみを持つ新しいデータフレームの作成
        new_df = DataFrame(
            race = race_group.race,
            box = race_group.box,
            normalized_odds = odds_processed.norm_prob,
            normalized_slits = norm_slits
        )

        push!(transformed_dfs, new_df)
    end

    # 全レースのデータを結合
    final_df = vcat(transformed_dfs...)

    # 新しいCSVファイルとして保存
    CSV.write("../data/race_nz.csv", final_df)

    return final_df
end

# 実行
processed_data = main()
println("処理が完了しました。新しいデータは'race_nz.csv'に保存されました。")

処理が完了しました。新しいデータは'race_nz.csv'に保存されました。
